<a href="https://colab.research.google.com/github/iwaohig/LINEWORKS-OAuth-Mail-Notifier/blob/main/%E6%9C%AA%E8%AA%AD%E3%83%A1%E3%83%BC%E3%83%AB%E3%82%92%E3%83%88%E3%83%BC%E3%82%AF%E3%81%A7%E9%80%81%E4%BF%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update # 更新
!apt install chromium-chromedriver # ChromeDriverのインストール
!cp /usr/lib/chromium-browser/chromedriver /usr/bin # パスへのコピー
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver') # Pythonのパスに追加

In [2]:
CLIENT_ID = '' #@param {type:"string"}
CLIENT_SECRET = '' #@param {type:"string"}
REDIRECT_URI = '' #@param {type:"string"}
SCOPE = '' #@param {type:"string"}
STATE = '' #@param {type:"string"}
LOGIN_ID = '' #@param {type:"string"}
PASSWORD = '' #@param {type:"string"}

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoAlertPresentException
import sys
import time
import requests
import urllib.parse

# OAuth認証のためのURL構築
AUTH_BASE_URL = 'https://auth.worksmobile.com/oauth2/v2.0/authorize'
auth_params = {
    'client_id': CLIENT_ID,
    'redirect_uri': REDIRECT_URI,
    'scope': SCOPE,
    'response_type': 'code',
    'state': STATE
}
auth_query = urllib.parse.urlencode(auth_params)
auth_url = f'{AUTH_BASE_URL}?{auth_query}'

# WebDriverの設定
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

# LINE WORKSログインページにアクセス
driver.get(auth_url)

# ユーザーIDとパスワードの入力
driver.find_element(By.ID, "inputId").send_keys(LOGIN_ID)
driver.find_element(By.ID, "password").send_keys(PASSWORD)

# ログイン処理の実行
driver.execute_script("login()")

# リダイレクトを待つ
time.sleep(5)

# アラートが表示されているかを確認
try:
    alert = driver.switch_to.alert
    alert_text = alert.text
    alert.accept()
    print(f"Alert detected: {alert_text}")

except NoAlertPresentException:
    pass

# リダイレクト後のURLからAuthorization Codeを取得
auth_code = driver.current_url.split('code=')[1].split('&')[0]

# WebDriverを終了
driver.quit()

# Access Tokenの取得
token_url = 'https://auth.worksmobile.com/oauth2/v2.0/token'
data = {
    'code': auth_code,
    'grant_type': 'authorization_code',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
    'redirect_uri': REDIRECT_URI
}
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json().get('access_token')
    print("Access Token:", access_token)
else:
    print("Error fetching Access Token:")
    print(f"HTTP Status Code: {response.status_code}")
    print(f"Response: {response.json()}")

In [ ]:
import requests

# LINE WORKS API設定
base_url = "https://www.worksapis.com/v1.0"
user_id = "me"  # ユーザーID、ログインID、または"me"

# ヘッダーに認証情報を含める
headers = {
    "Authorization": f"Bearer {access_token}"
}

# メールフォルダリストを取得するAPIエンドポイント
url = f"{base_url}/users/{user_id}/mail/mailfolders"

# APIリクエストを実行
response = requests.get(url, headers=headers)

# レスポンスの確認
if response.status_code == 200:
    mail_folders = response.json().get('mailFolders', [])
    for folder in mail_folders:
        # 受信トレイのフォルダーIDを表示
        if folder.get('folderName') == "受信トレイ":
            print(f"受信トレイのフォルダーID: {folder.get('folderId')}")
else:
    print(f"エラー: {response.status_code}")

In [5]:
from bs4 import BeautifulSoup
import requests
import json

def html_to_text(html):
    """HTMLをプレーンテキストに変換する"""
    soup = BeautifulSoup(html, 'html.parser')
    return soup.get_text()

# LINE WORKS API設定
base_url = "https://www.worksapis.com/v1.0"
bot_api_base_url = "https://www.worksapis.com/v1.0"
user_id = "me"  # ユーザーID、ログインID、または"me"
folder_id = 0  # 受信トレイのフォルダID
bot_id = 6885052  # Bot ID
channel_id = "0c3a2bb0-ab63-0bfc-541f-44cf8ce5168e"  # トークルームID

# ヘッダーに認証情報を含める
headers = {"Authorization": f"Bearer {access_token}"}

# メールリスト取得APIエンドポイント
mail_list_url = f"{base_url}/users/{user_id}/mail/mailfolders/{folder_id}/children"

# メールリスト取得リクエスト
response = requests.get(mail_list_url, headers=headers)

if response.status_code == 200:
    mails = response.json().get('mails', [])
    unread_mail_ids = [mail['mailId'] for mail in mails if mail.get('status') == 'Unread']

    # 未読メールの詳細取得とトーク送信
    for mail_id in unread_mail_ids:
        mail_detail_url = f"{base_url}/users/{user_id}/mail/{mail_id}"
        detail_response = requests.get(mail_detail_url, headers=headers)

        if detail_response.status_code == 200:
            mail_detail = detail_response.json()
            # HTML本文をテキストに変換
            text_body = html_to_text(mail_detail['mail']['body'])
            subject = mail_detail['mail']['subject']
            message_content = {
                "type": "text",
                "text": f"件名: {subject}\n本文: {text_body}"
            }
            # メッセージ送信APIエンドポイント
            #send_message_url = f"{bot_api_base_url}/bots/{bot_id}/channels/{channel_id}/messages"
            send_message_url = f"{bot_api_base_url}/bots/{bot_id}/users/iwaoh%40lytest1.wmjse.tokyo/messages"

            # ヘッダーに 'Content-Type' を追加
            headers['Content-Type'] = 'application/json'

            # メッセージ送信
            message_response = requests.post(send_message_url, headers=headers, data=json.dumps({"content": message_content}))

            if message_response.status_code != 201:
                print(f"メールID {mail_id} のメッセージ送信に失敗: {message_response.status_code}")
        else:
            print(f"メールID {mail_id} の取得に失敗: {detail_response.status_code}")

else:
    print(f"メールリスト取得エラー: {response.status_code}")